Цель: привести две набора данных: провести эксперименты с моделями (lstm, lstm_bidir, ATI-CNN_ML, M_MBLF_ML, DNN_ML) c предобработкой и без предобработки. Сначала проведем эксперименты с моделями на непредобработанных данных, а потом на предобработанных данных. Предобработка данных будет заключаться в наработках Неймарка и мб ещё что-нибудь. В роли данных будут выступать 'all', возможно ещё 'diag', а также менее вероятно 'superdiag' и 'subdiag'.

Сначала попробуем на google collab.

### Для Google Colaboratory

In [ ]:
# Подключение Google Drive к виртуальной машине
from google.colab import drive
drive.mount('/content/drive')

# Копирование данных с Google Drive на локальный диск виртуальной машины.
!cp -r /content/drive/MyDrive/practice_2022-2023/data/ptbxlnpy/ .

### Импорт библиотек, определение функций и т.п.

In [3]:
# Для работы с данными
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
import tensorflow as tf
import math
import pickle

# Для работы с моделями
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split

# Для метрик
from keras import backend as K
from keras.metrics import AUC, Recall, Precision, Accuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import auc, roc_curve, classification_report
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

# Код из учебника Жерона 
K = keras.backend

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_batch_end(self, batch, logs):
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(logs["loss"])
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)

def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=10**-5, max_rate=10):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = np.exp(np.log(max_rate / min_rate) / iterations)
    init_lr = K.get_value(model.optimizer.learning_rate)
    K.set_value(model.optimizer.learning_rate, min_rate)
    exp_lr = ExponentialLearningRate(factor)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size,
                        callbacks=[exp_lr])
    K.set_value(model.optimizer.learning_rate, init_lr)
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses)
    plt.gca().set_xscale('log')
    plt.hlines(min(losses), min(rates), max(rates))
    plt.axis([min(rates), max(rates), min(losses), (losses[0] + min(losses)) / 2])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_epoch_begin(self, epoch, logs=None):
        self.prev_loss = 0
    def on_batch_end(self, batch, logs=None):
        batch_loss = logs["loss"] * (batch + 1) - self.prev_loss * batch
        self.prev_loss = logs["loss"]
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(batch_loss)
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)

class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)
        
class attention(layers.Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

def ATI_CNN(num_classes, multi_label_classifier=False):
    activation = keras.activations.sigmoid if multi_label_classifier else keras.activations.softmax

    def Conv_block_1(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    def Conv_block_2(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    shape = (1000, 12)
    input = keras.Input(shape=shape, name="ECG")

    x = Conv_block_1(64)(input)
    x = Conv_block_1(128)(x)

    x = Conv_block_2(256)(x)
    x = Conv_block_2(256)(x)
    x = Conv_block_2(256)(x)

    x = layers.LSTM(units=32, return_sequences=True)(x)
    x = layers.LSTM(units=32, return_sequences=True)(x)

    x = attention()(x)

    output = layers.Dense(units=num_classes, activation=activation, name='output')(x)

    model = keras.Model(inputs=[input], outputs=[output])

    return model

def M_ATI_CNN(num_classes, multi_label_classifier=False):
    activation = layers.Activation(keras.activations.sigmoid) if multi_label_classifier else layers.Activation(keras.activations.softmax)

    def Conv_block_1(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    def Conv_block_2(filters):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv1D(filters=filters, kernel_size=3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling1D(pool_size=3, strides=3),
            layers.Dropout(rate=0.2)
        ])

    shape = (1000, 12)
    input = keras.Input(shape=shape, name="ECG")

    output_modules = list()

    for i in range(num_classes):
      output_modules.append(keras.Sequential([
          Conv_block_1(64),
          Conv_block_1(128),
          Conv_block_2(256),
          Conv_block_2(256),
          Conv_block_2(256)
      ])(input))

      output_modules[-1] = layers.Bidirectional(layers.LSTM(units=32, return_sequences=True))(output_modules[-1])

      output_modules[-1] = attention()(output_modules[-1])
      output_modules[-1] = layers.Dense(units=1)(output_modules[-1])

    cat_outputs =  keras.layers.Concatenate()(output_modules)

    output = activation(cat_outputs)

    model = keras.Model(inputs=[input], outputs=[output])

    return model

def DNN(num_classes, multi_label_classifier=False):
    activation = keras.activations.sigmoid if multi_label_classifier else keras.activations.softmax

    def Conv_block_1(filters, kernel_size, pool_size):
        return keras.Sequential([
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(rate=0.1),
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.MaxPooling1D(pool_size=pool_size)
        ]), layers.MaxPooling1D(pool_size=pool_size)

    def Conv_block_2(filters, kernel_size, pool_size):
        return keras.Sequential([
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(rate=0.1),
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(rate=0.1),
            layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.MaxPooling1D(pool_size=pool_size)
        ]), layers.MaxPooling1D(pool_size=pool_size)

    shape = (1000, 12)
    input = keras.Input(shape=shape, name="ECG")

    x = keras.Sequential([
        layers.Conv1D(filters=32, kernel_size=16, padding='same'),
        layers.BatchNormalization(),
        layers.ReLU()
    ])(input)

    x1, x2 = Conv_block_1(kernel_size=16, filters=32, pool_size=2)
    x = layers.Concatenate()([x1(x), x2(x)])

    for filters, kernel_size, pool_size in zip(
        [32, 32, 32, 64, 64, 64, 64, 96, 96, 96, 96],
        [16, 16, 16, 16, 16,  8,  8,  8,  8,  8,  8],
        [ 1,  2,  1,  2,  1,  2,  1,  2,  1,  2,  1]
    ):
        x1, x2 = Conv_block_2(filters=filters, kernel_size=kernel_size, pool_size=pool_size)
        x = layers.Concatenate()([x1(x), x2(x)])

    x = keras.Sequential([
        layers.BatchNormalization(),
        layers.ReLU()
    ])(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True), merge_mode='sum')(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Dense(units=64, activation=keras.activations.relu)(x)
    output = layers.Dense(units=num_classes, activation=activation, name='output')(x)

    model = keras.Model(inputs=[input], outputs=[output])

    return model
        
# Загрузка ptbxl
def load_ptbxl(task):
  if task == 'diag':
    X_train = np.load('X_train_ptbxl_diag.npy')
    y_train = np.load('y_train_ptbxl_diag.npy')
    X_test = np.load('X_val_ptbxl_diag.npy')
    y_test = np.load('y_val_ptbxl_diag.npy')
  elif task == 'superdiag':
    X_train = np.load('X_train_ptbxl_superdiag.npy')
    y_train = np.load('y_train_ptbxl_superdiag.npy')
    X_test = np.load('X_val_ptbxl_superdiag.npy')
    y_test = np.load('y_val_ptbxl_superdiag.npy')
  elif task == 'all':
    X_train = np.load('data_npy/ptbxlnpy/X_train_ptbxl_all.npy')
    y_train = np.load('data_npy/ptbxlnpy/y_train_ptbxl_all.npy')
    X_test = np.load('data_npy/ptbxlnpy/X_val_ptbxl_all.npy')
    y_test = np.load('data_npy/ptbxlnpy/y_val_ptbxl_all.npy')
  elif task == 'subdiag':
    X_train = np.load('X_train_ptbxl_subdiag.npy')
    y_train = np.load('y_train_ptbxl_subdiag.npy')
    X_test = np.load('X_val_ptbxl_subdiag.npy')
    y_test = np.load('y_val_ptbxl_subdiag.npy')
  return X_train, y_train, X_test, y_test

# Сериализация
def sp(obj, name): # sp = serialization_pickle ;name = *.pkl
    with open(name, 'wb') as pickle_out:
        pickle.dump(obj, pickle_out)

# Компиляция и обучение модели
def AUC_Keras(y_true, y_pred):
    auc = keras.metrics.AUC(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

# Компиляция и обучение модели
def compile_fit(model, X_train, y_train, X_val = None, y_val = None, validation_split = 0.0, batch_size = None, epochs = None, early_stopping = None, model_checkpoint = None, onecycle = None):
  model.compile(loss = keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.experimental.AdamW(),
                metrics=['AUC'])
  
  if X_val == None:
    history = model.fit(X_train, y_train, 
                        epochs = epochs, 
                        batch_size = batch_size,
                        validation_data = None, 
                        validation_split=validation_split, 
                        callbacks=[onecycle, model_checkpoint, early_stopping])
    sp(history, 'history.pkl')
  else:
    history = model.fit(X_train, y_train, 
                        epochs = epochs,
                        batch_size = batch_size, 
                        validation_data = (X_val, y_val), 
                        validation_split=0.0, 
                        callbacks=[onecycle, model_checkpoint, early_stopping])
    sp(history, 'history.pkl')
  return history

# TP TN FP FN
def tp_tn_fp_fn(y_true, y_pred):
  TP = TruePositives()
  TN = TrueNegatives()
  FP = FalsePositives()
  FN = FalseNegatives()
  TP.update_state(y_true, y_pred)
  TN.update_state(y_true, y_pred)
  FP.update_state(y_true, y_pred)
  FN.update_state(y_true, y_pred)
  return TP.result().numpy(),  TN.result().numpy(),  FP.result().numpy(), FN.result().numpy() 

# Подсчет метрик
def calc_metrics(t, p, flag = 0): # t - y_true, p - y_pred
  y_true=np.argmax(t, axis=1)
  y_pred=np.argmax(p, axis=1)
  beta = 2

  f2_score = fbeta_score(y_true, y_pred, average='macro', beta=2)
  precision = precision_score(y_true, y_pred, average='macro')
  recall = recall_score(y_true, y_pred, average='macro')
  TP, TN, FP, FN = tp_tn_fp_fn(t, p)
  g2_score = TP/(TP+FP+beta*FN)
    
  sp(f2_score, 'f2_score.pkl')
  sp(g2_score, 'g2_score.pkl')
  sp(precision, 'precision.pkl')
  sp(recall, 'recall.pkl')
  sp(TP, 'TP.pkl')
  sp(TN, 'TN.pkl')
  sp(FP, 'FP.pkl')
  sp(FN, 'FN.pkl')
  sp(y_true, 'y_true_argmax.pkl')
  sp(y_pred, 'y_pred_argmax.pkl')
    
  if flag == 0:
    return f2_score, g2_score
  elif flag == 1:
    return f2_score, g2_score, precision, recall

# Таблица результатов
table_res_ptbxl = pd.DataFrame(columns = ('AUC', 'F2', 'G2'))

# Занесение новых результатов в таблицу
def edit_table(table, model, X, y, index_name): # X - X_test, y - y_test 
  score = model.evaluate(X, y)
  y_pr = model.predict(X)
  f2_score, g2_score = calc_metrics(y, y_pr, flag = 0)
  list_metrics = [score[1], f2_score, g2_score] # AUC, F2, G2
  table.loc[index_name] = list_metrics

  sp(score, 'score.pkl')
  sp(y_pr, 'y_pred.pkl')
  sp(y, 'y_test.pkl')
    
  return table

# График loss и accuracy
def plot_loss_and_accuracy_curves(_history):
  fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18,6))
  axs[0].plot(_history.history['loss'], color='b', label='Training loss')
  axs[0].plot(_history.history['val_loss'], color='r', label='Validation loss')
  axs[0].set_title("Loss")
  axs[0].legend(loc='best', shadow=True)
  axs[1].plot(_history.history['auc'], color='b', label='Training accuracy')
  axs[1].plot(_history.history['val_auc'], color='r', label='Validation accuracy')
  axs[1].set_title("AUC")
  axs[1].legend(loc='best', shadow=True)
  plt.show()
  
# Работа с моделями lstm и lstm_bidir
def type_comp_fit_save_model_score(table, X_train, y_train, X_test, y_test, type_model, save_name, index_model_task, batch_size, epochs, onecycle):
  # количество классов
  num_classes = y_train.shape[1]
  sp(num_classes, 'num_classes.pkl')

  # Выбор архитектуры модели
  if type_model == 'lstm':
    inputs = keras.Input(shape=(1000, 12))
    x = layers.LSTM(units=256,
                    return_sequences=True,
                    stateful=False,
                    unroll=False)(inputs)
    x = layers.LeakyReLU()(x)
    x = layers.LSTM(units=256,
                    return_sequences=True, # True if concat
                    stateful=False,
                    unroll=False)(x)
    x = layers.LeakyReLU()(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    concat = layers.Concatenate()([avg_pool, max_pool])
    outputs = layers.Dense(units=num_classes, activation='sigmoid')(concat)
    model = keras.Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    
    sp(model, 'model.pkl')
    sp(model.summary(), 'model_summary().pkl')
    
    # Реализация раннего прекращения.
    checkpoint_filepath = './checkpoint_lstm/'
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                          save_weights_only=True,
                                                          save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=epochs//10,
                                                  restore_best_weights=True)
  elif type_model == 'lstm_bidir':
    inputs = keras.Input(shape=(1000, 12))
    x = layers.Bidirectional(layers.LSTM(units=256,
                             return_sequences = True,
                             stateful = False,
                             unroll = False))(inputs)
    x = layers.LeakyReLU()(x)
    x = layers.Bidirectional(layers.LSTM(units=256,
                             return_sequences = True,
                             stateful = False,
                             unroll = False))(x)
    x = layers.LeakyReLU()(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    concat = layers.Concatenate()([avg_pool, max_pool])
    outputs = layers.Dense(units=num_classes, activation='sigmoid')(concat)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.build(input_shape = (None, 1000, 12)) # `input_shape` is the shape of the input data
    
    print(model.summary())

    # Реализация раннего прекращения.
    checkpoint_filepath = './checkpoint_lstm_bidir/'
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                          save_weights_only=True,
                                                          save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=epochs//10,
                                                  restore_best_weights=True)
  
  # Обучение
  History = compile_fit(model, X_train, y_train, validation_split=0.1, batch_size=batch_size, epochs=epochs, early_stopping=early_stopping, model_checkpoint=model_checkpoint, onecycle=onecycle)

  # Сохранение модели
  model.save(save_name)

  # Построение графика
  plot_loss_and_accuracy_curves(History)

  # Сохранение в таблицу
  table = edit_table(table, model, X_test, y_test, index_model_task)

  sp(table, 'table.pkl')
  return table
                        
tf.random.set_seed(42)
%matplotlib inline

### Модель lstm на категории "all"

In [ ]:
X_train, y_train, X_test, y_test = load_ptbxl(task = 'all')
batch_size = 512
epochs = 200
onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * epochs, max_rate=0.0001)
table_res_ptbxl = type_comp_fit_save_model_score(table_res_ptbxl, X_train, y_train, X_test, y_test,type_model = 'lstm', save_name = 'model_lstm_all',index_model_task = 'lstm_all', batch_size=batch_size, epochs=epochs, onecycle=onecycle)
del(X_train)
del(y_train)
del(X_test)
del(y_test)

### Модель lstm_bidir на категории "all"

In [ ]:
X_train, y_train, X_test, y_test = load_ptbxl(task = 'all')
batch_size = 256
epochs = 200
onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * epochs, max_rate=0.001)
table_res_ptbxl = type_comp_fit_save_model_score(table_res_ptbxl, X_train, y_train, X_test, y_test,type_model = 'lstm_bidir', save_name = 'model_lstm_bidir_all',index_model_task = 'lstm_bidir_all', batch_size=batch_size, epochs=epochs, onecycle=onecycle)
del(X_train)
del(y_train)
del(X_test)
del(y_test)

### Модель ATI-CNN_ML на категории "all"

In [ ]:
model = ATI_CNN(num_classes=5, multi_label_classifier=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=[keras.losses.BinaryCrossentropy()],
    metrics=[keras.metrics.AUC()]
)

callbacks = [
    keras.callbacks.EarlyStopping(
        # Прекратить обучение если `val_loss` больше не улучшается
        monitor='val_auc',
        mode='max',
        # "больше не улучшается" определим как "не лучше чем min_delta и меньше"
        min_delta=1e-2,
        # "больше не улучшается" далее определим как "как минимум в течение patience эпох"
        patience=5,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='drive/MyDrive/sciwork/ATI_CNN_ML.h5',
        # Путь по которому нужно сохранить модель
        # Два параметра ниже значат что мы перезапишем
        # текущий чекпоинт в том и только в том случае, когда
        # улучится значение `val_loss`.
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
]

history = model.fit(X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=64,
        callbacks=callbacks,
        shuffle=True)

### Модель M_MBLF_ML на категории "all"

In [ ]:
model = M_ATI_CNN(num_classes=5, multi_label_classifier=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=[keras.losses.BinaryCrossentropy()],
    metrics=[keras.metrics.AUC()]
)

callbacks = [
    keras.callbacks.EarlyStopping(
        # Прекратить обучение если `val_loss` больше не улучшается
        monitor='val_auc',
        mode='max',
        # "больше не улучшается" определим как "не лучше чем min_delta и меньше"
        min_delta=1e-2,
        # "больше не улучшается" далее определим как "как минимум в течение patience эпох"
        patience=5,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='drive/MyDrive/sciwork/M_ATI_CNN_ML.h5',
        # Путь по которому нужно сохранить модель
        # Два параметра ниже значат что мы перезапишем
        # текущий чекпоинт в том и только в том случае, когда
        # улучится значение `val_loss`.
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
]

history = model.fit(X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=64,
        callbacks=callbacks,
        shuffle=True)

### Модель DNN_ML на категории "all"

In [ ]:
model = DNN(num_classes=5, multi_label_classifier=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=[keras.losses.BinaryCrossentropy()],
    metrics=[keras.metrics.AUC()]
)

callbacks = [
    keras.callbacks.EarlyStopping(
        # Прекратить обучение если `val_loss` больше не улучшается
        monitor='val_auc',
        mode='max',
        # "больше не улучшается" определим как "не лучше чем min_delta и меньше"
        min_delta=1e-2,
        # "больше не улучшается" далее определим как "как минимум в течение patience эпох"
        patience=5,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='drive/MyDrive/sciwork/DNN_ML.h5',
        # Путь по которому нужно сохранить модель
        # Два параметра ниже значат что мы перезапишем
        # текущий чекпоинт в том и только в том случае, когда
        # улучится значение `val_loss`.
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
]

history = model.fit(X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=64,
        callbacks=callbacks,
        shuffle=True)